## 8.1.- Joins en PySpark

In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf = conf)

21/09/28 10:12:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
x = sc.parallelize([("a", 5), ("b", 6), ("c", 7), ("d", 8)])
y = sc.parallelize([("a", 1), ("a", 2), ("c", 3)])

In [6]:
x.join(y).collect()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('c', (7, 3)), ('a', (5, 1)), ('a', (5, 2))]

In [7]:
y.join(x).collect()

[('c', (3, 7)), ('a', (1, 5)), ('a', (2, 5))]

In [8]:
y.leftOuterJoin(x).collect()

[('c', (3, 7)), ('a', (1, 5)), ('a', (2, 5))]

In [9]:
x.leftOuterJoin(y).collect()

[('b', (6, None)),
 ('c', (7, 3)),
 ('d', (8, None)),
 ('a', (5, 1)),
 ('a', (5, 2))]

In [10]:
y.rightOuterJoin(x).collect()

[('c', (3, 7)),
 ('b', (None, 6)),
 ('d', (None, 8)),
 ('a', (1, 5)),
 ('a', (2, 5))]

## 8.2.- Acumuladores. Cómo detectar patrones en los datos

In [11]:
lines = sc.textFile("ejemplo.txt")

In [12]:
py = sc.accumulator(0)
sp = sc.accumulator(0)

In [13]:
def lenguajes(linea):
    global py, sp
    if "Python" in linea:
        py += 1
        if "Spark" in linea:
            sp += 1
        return True
    if "Spark" in linea:
        sp += 1
        return True
    else:
        return False
    
valores = lines.filter(lenguajes)

In [14]:
valores.collect()

['Apache Spark',
 'Apache Spark es un framework de computación (entorno de trabajo) en clúster open-source. Fue desarrollada originariamente en la Universidad de California, en el AMPLab de Berkeley. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. Spark proporciona una interfaz para la programación de clusters completos con Paralelismo de Datos implícito y tolerancia a fallos.',
 'Apache Spark se puede considerar un sistema de computación en clúster de propósito general y orientado a la velocidad. Proporciona APIs en Java, Scala, Python y R. También proporciona un motor optimizado que soporta la ejecución de grafos en general. También soporta un conjunto extenso y rico de herramientas de alto nivel entre las que se incluyen Spark SQL (para el procesamiento de datos estructurados basada en SQL), MLlib para implementar machine learning, GraphX para el procesamiento de grafos y Spark Streaming.',
 'S

In [15]:
py

Accumulator<id=0, value=1>

In [16]:
sp

Accumulator<id=1, value=18>

## 8.3.- Cómo construir funciones map

In [20]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext

conf = SparkConf().setMaster("local").setAppName("Mi programa")
#sc = SparkContext(conf = conf) ya se ha cargado previamente
sqlContext = SQLContext(sc)

dfspark = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("/Users/raqueldiaz/Documents/Linkedin Learning/Python para data science y big data esencial/base_datos_2008.csv")
dfspark = dfspark.sample(fraction=0.001, withReplacement=False)
dfspark = dfspark.withColumn("ArrDelay", dfspark["ArrDelay"].cast("integer"))

df2 = dfspark.na.drop(subset=["ArrDelay", "DepDelay", "Distance"])
df2 = df2.filter("ArrDelay is not NULL")
df2 = df2.dropDuplicates()

In [21]:
#"array" paralelizado de la columna "Origin"
A = sc.parallelize(df2.select("Origin").rdd.collect())

In [22]:
#se guarda en memoria
A.persist()

ParallelCollectionRDD[84] at readRDDFromFile at PythonRDD.scala:274

In [23]:
mapfunction = A.map(lambda x: (x,1))

In [24]:
mapfunction.collect()

[(Row(Origin='SNA'), 1),
 (Row(Origin='IAH'), 1),
 (Row(Origin='SJC'), 1),
 (Row(Origin='STT'), 1),
 (Row(Origin='CMH'), 1),
 (Row(Origin='ATL'), 1),
 (Row(Origin='LAX'), 1),
 (Row(Origin='MSP'), 1),
 (Row(Origin='BOS'), 1),
 (Row(Origin='DTW'), 1),
 (Row(Origin='XNA'), 1),
 (Row(Origin='MIA'), 1),
 (Row(Origin='MKE'), 1),
 (Row(Origin='LAS'), 1),
 (Row(Origin='SAN'), 1),
 (Row(Origin='DEN'), 1),
 (Row(Origin='PHX'), 1),
 (Row(Origin='BWI'), 1),
 (Row(Origin='LAX'), 1),
 (Row(Origin='TPA'), 1),
 (Row(Origin='MKE'), 1),
 (Row(Origin='DFW'), 1),
 (Row(Origin='DFW'), 1),
 (Row(Origin='JFK'), 1),
 (Row(Origin='OGG'), 1),
 (Row(Origin='LAX'), 1),
 (Row(Origin='CVG'), 1),
 (Row(Origin='MEM'), 1),
 (Row(Origin='STL'), 1),
 (Row(Origin='ATL'), 1),
 (Row(Origin='BWI'), 1),
 (Row(Origin='ACV'), 1),
 (Row(Origin='SLC'), 1),
 (Row(Origin='DFW'), 1),
 (Row(Origin='BDL'), 1),
 (Row(Origin='SDF'), 1),
 (Row(Origin='BWI'), 1),
 (Row(Origin='SFO'), 1),
 (Row(Origin='LIH'), 1),
 (Row(Origin='BOS'), 1),


In [25]:
def fun(x):
    if x[0] in ["SEA", "ATL", "HOU"]:
        return((x,2))
    elif x[0] == "DEN":
        return((x,3))
    else:
        return((x,1))

In [26]:
mapfunction2 = A.map(fun)

In [27]:
mapfunction2.collect()

[(Row(Origin='SNA'), 1),
 (Row(Origin='IAH'), 1),
 (Row(Origin='SJC'), 1),
 (Row(Origin='STT'), 1),
 (Row(Origin='CMH'), 1),
 (Row(Origin='ATL'), 2),
 (Row(Origin='LAX'), 1),
 (Row(Origin='MSP'), 1),
 (Row(Origin='BOS'), 1),
 (Row(Origin='DTW'), 1),
 (Row(Origin='XNA'), 1),
 (Row(Origin='MIA'), 1),
 (Row(Origin='MKE'), 1),
 (Row(Origin='LAS'), 1),
 (Row(Origin='SAN'), 1),
 (Row(Origin='DEN'), 3),
 (Row(Origin='PHX'), 1),
 (Row(Origin='BWI'), 1),
 (Row(Origin='LAX'), 1),
 (Row(Origin='TPA'), 1),
 (Row(Origin='MKE'), 1),
 (Row(Origin='DFW'), 1),
 (Row(Origin='DFW'), 1),
 (Row(Origin='JFK'), 1),
 (Row(Origin='OGG'), 1),
 (Row(Origin='LAX'), 1),
 (Row(Origin='CVG'), 1),
 (Row(Origin='MEM'), 1),
 (Row(Origin='STL'), 1),
 (Row(Origin='ATL'), 2),
 (Row(Origin='BWI'), 1),
 (Row(Origin='ACV'), 1),
 (Row(Origin='SLC'), 1),
 (Row(Origin='DFW'), 1),
 (Row(Origin='BDL'), 1),
 (Row(Origin='SDF'), 1),
 (Row(Origin='BWI'), 1),
 (Row(Origin='SFO'), 1),
 (Row(Origin='LIH'), 1),
 (Row(Origin='BOS'), 1),


## 8.4.- Cómo construir funciones reduce

In [28]:
reducefunction = mapfunction.reduceByKey(lambda x,y: x+y)

In [29]:
reducefunction.collect()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[(Row(Origin='SNA'), 54),
 (Row(Origin='IAH'), 167),
 (Row(Origin='SJC'), 50),
 (Row(Origin='STT'), 6),
 (Row(Origin='CMH'), 31),
 (Row(Origin='ATL'), 409),
 (Row(Origin='LAX'), 187),
 (Row(Origin='MSP'), 138),
 (Row(Origin='BOS'), 121),
 (Row(Origin='DTW'), 153),
 (Row(Origin='XNA'), 10),
 (Row(Origin='MIA'), 61),
 (Row(Origin='MKE'), 50),
 (Row(Origin='LAS'), 194),
 (Row(Origin='SAN'), 97),
 (Row(Origin='DEN'), 250),
 (Row(Origin='PHX'), 209),
 (Row(Origin='BWI'), 90),
 (Row(Origin='TPA'), 73),
 (Row(Origin='DFW'), 278),
 (Row(Origin='JFK'), 94),
 (Row(Origin='OGG'), 19),
 (Row(Origin='CVG'), 78),
 (Row(Origin='MEM'), 76),
 (Row(Origin='STL'), 71),
 (Row(Origin='ACV'), 3),
 (Row(Origin='SLC'), 140),
 (Row(Origin='BDL'), 26),
 (Row(Origin='SDF'), 30),
 (Row(Origin='SFO'), 152),
 (Row(Origin='LIH'), 14),
 (Row(Origin='SEA'), 113),
 (Row(Origin='BRO'), 2),
 (Row(Origin='DCA'), 90),
 (Row(Origin='ORD'), 358),
 (Row(Origin='ABQ'), 47),
 (Row(Origin='PHL'), 99),
 (Row(Origin='BUR'), 30),
 

In [30]:
reducefunction.take(10)

[(Row(Origin='SNA'), 54),
 (Row(Origin='IAH'), 167),
 (Row(Origin='SJC'), 50),
 (Row(Origin='STT'), 6),
 (Row(Origin='CMH'), 31),
 (Row(Origin='ATL'), 409),
 (Row(Origin='LAX'), 187),
 (Row(Origin='MSP'), 138),
 (Row(Origin='BOS'), 121),
 (Row(Origin='DTW'), 153)]

In [31]:
reducefunction2 = mapfunction2.reduceByKey(lambda x,y: x+y)

In [32]:
reducefunction2.take(10)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[(Row(Origin='SNA'), 54),
 (Row(Origin='IAH'), 167),
 (Row(Origin='SJC'), 50),
 (Row(Origin='STT'), 6),
 (Row(Origin='CMH'), 31),
 (Row(Origin='ATL'), 818),
 (Row(Origin='LAX'), 187),
 (Row(Origin='MSP'), 138),
 (Row(Origin='BOS'), 121),
 (Row(Origin='DTW'), 153)]

In [33]:
reducefunction.sortByKey().take(10)

[(Row(Origin='ABE'), 2),
 (Row(Origin='ABI'), 1),
 (Row(Origin='ABQ'), 47),
 (Row(Origin='ABY'), 1),
 (Row(Origin='ACT'), 2),
 (Row(Origin='ACV'), 3),
 (Row(Origin='ADQ'), 1),
 (Row(Origin='AEX'), 3),
 (Row(Origin='ALB'), 13),
 (Row(Origin='ALO'), 1)]

In [34]:
reducefunction.sortBy(lambda x: x[1], ascending=False).take(10)

[(Row(Origin='ATL'), 409),
 (Row(Origin='ORD'), 358),
 (Row(Origin='DFW'), 278),
 (Row(Origin='DEN'), 250),
 (Row(Origin='PHX'), 209),
 (Row(Origin='LAS'), 194),
 (Row(Origin='LAX'), 187),
 (Row(Origin='IAH'), 167),
 (Row(Origin='DTW'), 153),
 (Row(Origin='SFO'), 152)]

In [35]:
reducefunction2.sortBy(lambda x: x[1], ascending=False).take(10)

[(Row(Origin='ATL'), 818),
 (Row(Origin='DEN'), 750),
 (Row(Origin='ORD'), 358),
 (Row(Origin='DFW'), 278),
 (Row(Origin='SEA'), 226),
 (Row(Origin='PHX'), 209),
 (Row(Origin='LAS'), 194),
 (Row(Origin='LAX'), 187),
 (Row(Origin='IAH'), 167),
 (Row(Origin='DTW'), 153)]

## 8.5.- Ejemplos de MapReduce en PySpark

In [36]:
lines.getNumPartitions()

1

In [37]:
py = sc.accumulator(0)
sp = sc.accumulator(0)

def lenguajes(linea):
    global py, sp
    if "Python" in linea:
        py += 1
        if "Spark" in linea:
            sp += 1
        return True
    if "Spark" in linea:
        sp += 1
        return True
    else:
        return False
    
valores = lines.filter(lenguajes)

In [38]:
valores.count()

18

In [39]:
py

Accumulator<id=2, value=1>

In [40]:
sp

Accumulator<id=3, value=18>

In [41]:
funcionmap = valores.map(lambda x: (x,1))

In [43]:
contarvalores = funcionmap.reduceByKey(lambda x,y: x+y)

In [44]:
contarvalores.count()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


18

In [45]:
contarvalores.sortBy(lambda x: x[1], ascending=False).take(5)

[('Apache Spark', 1),
 ('Apache Spark es un framework de computación (entorno de trabajo) en clúster open-source. Fue desarrollada originariamente en la Universidad de California, en el AMPLab de Berkeley. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. Spark proporciona una interfaz para la programación de clusters completos con Paralelismo de Datos implícito y tolerancia a fallos.',
  1),
 ('Apache Spark se puede considerar un sistema de computación en clúster de propósito general y orientado a la velocidad. Proporciona APIs en Java, Scala, Python y R. También proporciona un motor optimizado que soporta la ejecución de grafos en general. También soporta un conjunto extenso y rico de herramientas de alto nivel entre las que se incluyen Spark SQL (para el procesamiento de datos estructurados basada en SQL), MLlib para implementar machine learning, GraphX para el procesamiento de grafos y Spark Str

In [46]:
def lenguajes_map(x):
    if "Python" in x and "Spark" in x:
        return ("Count", (1,1))
    elif "Python" in x:
        return ("Count", (1,0))
    elif "Spark" in x:
        return ("Count", (0,1))
    else:
        return ("Count", (0,0))
mapfun = lines.map(lenguajes_map)

In [47]:
mapfun.count()

71

In [49]:
lines.count()

71

In [48]:
mapfun.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).collect()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('Count', (1, 18))]